In [0]:
%sql
CREATE TABLE fq_dev_catalog.silver.dim_brand
(
    sys_id STRING NOT NULL,
    brand_name STRING NOT NULL,
    company_id STRING NOT NULL,
    is_active BOOLEAN NOT NULL,
    source_system STRING NOT NULL,

    CONSTRAINT brand_name_pk PRIMARY KEY (brand_name),
    CONSTRAINT fk_company
        FOREIGN KEY (company_id)
        REFERENCES fq_dev_catalog.silver.dim_company(company_id)
)
USING DELTA
TBLPROPERTIES (
    delta.enableChangeDataFeed = true
);




In [0]:
%sql
select * from fq_dev_catalog.silver.dim_brand

In [0]:
%sql
select * from fq_dev_catalog.silver.dim_company

In [0]:
%sql
SELECT company_id, company_name
FROM fq_dev_catalog.silver.dim_company;


In [0]:
from pyspark.sql.functions import lit, col, expr

dim_company_df = (
    spark.read
        .table("fq_dev_catalog.silver.dim_company")
        .select("company_id")
)


In [0]:
brand_mapping_df = spark.createDataFrame(
    [
        ("ALBAIK", "SQ"),
        ("DENNYS", "AI")
    ],
    ["brand_name", "company_id"]
)


In [0]:
brand_enriched_df = (
    brand_mapping_df
        .join(
            dim_company_df,
            on="company_id",
            how="inner"   # INNER ensures FK safety
        )
)


In [0]:
brand_final_df = (
    brand_enriched_df
        .select(
            expr("uuid()").alias("sys_id"),
            col("brand_name"),
            col("company_id"),
            lit(True).alias("is_active"),
            lit("FQ").alias("source_system")
        )
)


In [0]:
brand_final_df.display()

In [0]:
(
    brand_final_df
        .write
        .format("delta")
        .mode("append")
        .saveAsTable("fq_dev_catalog.silver.dim_brand")
)


In [0]:
spark.sql("""
SELECT 
    b.brand_name,
    c.company_name,
    b.company_id
FROM fq_dev_catalog.silver.dim_brand b
JOIN fq_dev_catalog.silver.dim_company c
    ON b.company_id = c.company_id
""").display()


In [0]:
df_brand_existing = spark.read.table("fq_dev_catalog.silver.dim_brand")
df_company = spark.read.table("fq_dev_catalog.silver.dim_company").select("company_id")

In [0]:
df_brand.display()

In [0]:
from pyspark.sql.functions import expr

df_new_brands = spark.createDataFrame(
    [("7", "ALBAIK", True, "FQ"), 
     ("6", "Dennys", True, "FQ")],
    ["brand_id", "brand_name", "is_active", "source_system"]
)

In [0]:
df_new_brands = df_new_brands.join(
    df_company, 
    df_new_brands.source_system == df_company.company_id,
    how="inner"
)

In [0]:
df_new_brands = df_new_brands.withColumn("sys_id", expr("uuid()"))


In [0]:
%sql
select * from fq_dev_catalog.silver.dim_brand

In [0]:
dim_brand_final=df_new_brands.select(
    "sys_id",
    "brand_id",
    "brand_name",
    "company_id",
    "is_active",
    "source_system"
)
dim_brand_final.display()

In [0]:
dim_brand_final.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("fq_dev_catalog.silver.dim_brand")
